In [21]:
from decouple import config

In [22]:
assert config("ALPHA_VANTAGE_API_KEY", default=None, cast=str) is not None

In [23]:
ALPHA_VANTAGE_API_KEY = config("ALPHA_VANTAGE_API_KEY", default=None, cast=str)

In [24]:
from datetime import datetime
import pytz
from decimal import Decimal

def transform_alpha_vantage_result(timestamp_str, result_val):
    #unix_timestamp = result.get('t')/1000
    #utc_timestamp = datetime.fromtimestamp(unix_timestamp, tz=pytz.timezone("UTC"))
    #print(utc_timestamp)
    timestamp_format = "%Y-%m-%d %H:%M:%S"
    eastern = pytz.timezone("US/Eastern")
    utc = pytz.utc
    timestamp = eastern.localize(datetime.strptime(timestamp_str, timestamp_format)).astimezone(utc)
    # timestamp
    return{
        "open_price" : Decimal(result_val["1. open"]),
        "close_price" : Decimal(result_val["4. close"]),
        "high_price" : Decimal(result_val["2. high"]),
        "low_price" : Decimal(result_val["3. low"]),
        "number_of_trades" : None,
        "volume" : int(result_val["5. volume"]),
        "volume_weighted_average" : None,
        "time" : timestamp,
    }

In [28]:
from dataclasses import dataclass
from typing import Literal
from urllib.parse import urlencode
import requests

ALPHA_VANTAGE_API_KEY = config("ALPHA_VANTAGE_API_KEY", default=None, cast=str)

@dataclass
class AlphaVantageAPIClient:
    ticker: str = "AAPL"  # company's symbol in stocks
    api_key: str = ""
    function: Literal["TIME_SERIES_INTRADAY"] = "TIME_SERIES_INTRADAY"
    interval: Literal["1min", "5min", "15min", "30min", "60min"] = "1min"
    month: str = "2024-11"
    

    def get_api_key(self):
        return self.api_key or ALPHA_VANTAGE_API_KEY
    
    def get_headers(self):
        api_key = self.get_api_key()
        return {}

    def get_params(self):
        return {
            "apikey" : self.get_api_key(),
            "symbol" : self.ticker,
            "function" : self.function,
            "interval" : self.interval,
            "month" : self.month
        }

    def generate_url(self, pass_auth=False):
        path = "/query"
        url = f"https://www.alphavantage.co{path}"
        params = self.get_params()
        encoded_params = urlencode(params)
        url = f"{url}?{encoded_params}"
        if pass_auth:
            api_key = self.get_api_key()
            url += f"&apiKey={api_key}"
        return url

    def fetch_data(self):
        headers = self.get_headers()
        url = self.generate_url()
        response = requests.get(url, headers=headers)
        response.raise_for_status() # not 200/201. Incase there is an error
        return response.json()

    def get_stock_data(self):
        data = self.fetch_data()
        dataset_key = [x for x in list(data.keys()) if not x.lower() == "meta data"][0]
        results = data[dataset_key]  # dictionary
        dataset = []
        for timestamp_str in results.keys():
            dataset.append(
                transform_alpha_vantage_result(timestamp_str, results.get(timestamp_str))
            )
        return dataset

In [29]:
stock_api_client = AlphaVantageAPIClient()
dataset = stock_api_client.get_stock_data()
dataset

[{'open_price': Decimal('237.3600'),
  'close_price': Decimal('237.4450'),
  'high_price': Decimal('237.4900'),
  'low_price': Decimal('237.3600'),
  'number_of_trades': None,
  'volume': 616,
  'volume_weighted_average': None,
  'time': datetime.datetime(2024, 11, 29, 21, 59, tzinfo=<UTC>)},
 {'open_price': Decimal('237.3750'),
  'close_price': Decimal('237.3600'),
  'high_price': Decimal('237.4900'),
  'low_price': Decimal('237.3535'),
  'number_of_trades': None,
  'volume': 598,
  'volume_weighted_average': None,
  'time': datetime.datetime(2024, 11, 29, 21, 58, tzinfo=<UTC>)},
 {'open_price': Decimal('237.3900'),
  'close_price': Decimal('237.3750'),
  'high_price': Decimal('237.3987'),
  'low_price': Decimal('237.3000'),
  'number_of_trades': None,
  'volume': 339,
  'volume_weighted_average': None,
  'time': datetime.datetime(2024, 11, 29, 21, 57, tzinfo=<UTC>)},
 {'open_price': Decimal('237.3250'),
  'close_price': Decimal('237.3000'),
  'high_price': Decimal('237.3900'),
  'low